In [1]:
import pandas as pd  
from sklearn.neighbors import KNeighborsClassifier
import pickle

In [5]:
df=pd.read_csv('Courier rounds.xlsx - Round_Schedules.csv')

In [6]:
df

,Index,Round ID,Vehicle ID,Time,Location,Postcode,Task
0,1,Rnd_01,Veh_01,7:30,Loc_101,SO30 2UT,Daily van check
1,2,Rnd_01,Veh_01,7:45,Loc_071,SO14 7NR,Deliver/Collect post
2,3,Rnd_01,Veh_01,7:50,Depot,SO14 5FE,Deliver/Collect post
3,4,Rnd_01,Veh_01,8:15,Loc_020,SO15 0AE,Deliver/Collect post
4,5,Rnd_01,Veh_01,8:20,Loc_028,SO15 0AE,Deliver/Collect post
...,...,...,...,...,...,...,...
95,96,Rnd_05,Veh_05,12:25,Loc_008,SO18 1NT,Collect dinners
96,97,Rnd_05,Veh_05,12:45,Loc_056,SO18 5NG,Deliver dinners
97,98,Rnd_05,Veh_05,13:05,Depot,SO14 5FE,Break/Charge van
98,99,Rnd_05,Veh_05,13:35,Spare time,NaN,Spare time


In [7]:
# Encoding for model
df['task_id'] = df['Task'].factorize()[0]
df['stop_id'] = df['Postcode'].factorize()[0]

In [8]:
df.head()

,Index,Round ID,Vehicle ID,Time,Location,Postcode,Task,task_id,stop_id
0,1,Rnd_01,Veh_01,7:30,Loc_101,SO30 2UT,Daily van check,0,0
1,2,Rnd_01,Veh_01,7:45,Loc_071,SO14 7NR,Deliver/Collect post,1,1
2,3,Rnd_01,Veh_01,7:50,Depot,SO14 5FE,Deliver/Collect post,1,2
3,4,Rnd_01,Veh_01,8:15,Loc_020,SO15 0AE,Deliver/Collect post,1,3
4,5,Rnd_01,Veh_01,8:20,Loc_028,SO15 0AE,Deliver/Collect post,1,3


In [9]:
df

,Index,Round ID,Vehicle ID,Time,Location,Postcode,Task,task_id,stop_id
0,1,Rnd_01,Veh_01,7:30,Loc_101,SO30 2UT,Daily van check,0,0
1,2,Rnd_01,Veh_01,7:45,Loc_071,SO14 7NR,Deliver/Collect post,1,1
2,3,Rnd_01,Veh_01,7:50,Depot,SO14 5FE,Deliver/Collect post,1,2
3,4,Rnd_01,Veh_01,8:15,Loc_020,SO15 0AE,Deliver/Collect post,1,3
4,5,Rnd_01,Veh_01,8:20,Loc_028,SO15 0AE,Deliver/Collect post,1,3
...,...,...,...,...,...,...,...,...,...
95,96,Rnd_05,Veh_05,12:25,Loc_008,SO18 1NT,Collect dinners,6,41
96,97,Rnd_05,Veh_05,12:45,Loc_056,SO18 5NG,Deliver dinners,4,40
97,98,Rnd_05,Veh_05,13:05,Depot,SO14 5FE,Break/Charge van,2,2
98,99,Rnd_05,Veh_05,13:35,Spare time,NaN,Spare time,10,-1


In [11]:
selected_columns = df[['Task', 'task_id']]
print(selected_columns)

                    Task  task_id
0        Daily van check        0
1   Deliver/Collect post        1
2   Deliver/Collect post        1
3   Deliver/Collect post        1
4   Deliver/Collect post        1
..                   ...      ...
95       Collect dinners        6
96       Deliver dinners        4
97      Break/Charge van        2
98            Spare time       10
99                Finish       11

[100 rows x 2 columns]


In [14]:
unique_task_lists = df.groupby('task_id')['Task'].apply(set).to_dict()

# Displaying the dictionary containing unique task lists for each task_id
print(unique_task_lists)

{0: {'Daily van check'}, 1: {'Deliver/Collect post'}, 2: {'Break/Charge van'}, 3: {'Collect dinners (x2)'}, 4: {'Deliver dinners'}, 5: {'Lunch'}, 6: {'Collect dinners'}, 7: {'Deliver dinners (x2)'}, 8: {'Collect samples'}, 9: {'Deliver samples'}, 10: {'Spare time'}, 11: {'Finish'}, 12: {'Daily Van Check'}, 13: {'Collect dinners '}, 14: {'Deliver dinners '}}


In [15]:
transformed_dict = {key: value.pop() for key, value in unique_task_lists.items()}
print(transformed_dict)

{0: 'Daily van check', 1: 'Deliver/Collect post', 2: 'Break/Charge van', 3: 'Collect dinners (x2)', 4: 'Deliver dinners', 5: 'Lunch', 6: 'Collect dinners', 7: 'Deliver dinners (x2)', 8: 'Collect samples', 9: 'Deliver samples', 10: 'Spare time', 11: 'Finish', 12: 'Daily Van Check', 13: 'Collect dinners ', 14: 'Deliver dinners '}


In [16]:
df['vehicle_id'] = df['Vehicle ID'].factorize()[0]

In [18]:
unique_veh_lists = df.groupby('vehicle_id')['Vehicle ID'].apply(set).to_dict()
print(unique_veh_lists)

{0: {'Veh_01'}, 1: {'Veh_02'}, 2: {'Veh_03'}, 3: {'Veh_04'}, 4: {'Veh_05'}}


In [19]:
df.head()

,Index,Round ID,Vehicle ID,Time,Location,Postcode,Task,task_id,stop_id,vehicle_id
0,1,Rnd_01,Veh_01,7:30,Loc_101,SO30 2UT,Daily van check,0,0,0
1,2,Rnd_01,Veh_01,7:45,Loc_071,SO14 7NR,Deliver/Collect post,1,1,0
2,3,Rnd_01,Veh_01,7:50,Depot,SO14 5FE,Deliver/Collect post,1,2,0
3,4,Rnd_01,Veh_01,8:15,Loc_020,SO15 0AE,Deliver/Collect post,1,3,0
4,5,Rnd_01,Veh_01,8:20,Loc_028,SO15 0AE,Deliver/Collect post,1,3,0


In [20]:
# Training data
X = df[['task_id', 'stop_id'] ]
y = df['vehicle_id']

In [28]:
X.columns = ['task_id', 'stop_id']

In [32]:
knn = KNeighborsClassifier()
knn.fit(X, y)

KNeighborsClassifier()

In [35]:
prediction = knn.predict([[5,3],[12,40]])

/Users/pujaagarwal/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [36]:
prediction

array([3, 4])